In [48]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from gensim.models import Word2Vec

In [73]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [35]:
import pickle

with open("../pickle_data/charlevel.pickle", "rb") as f:
    balenced_data = pickle.load(f)

In [2]:
balenced_data.keys()

dict_keys(['char_chat', 'idx2char', 'char2idx', 'target'])

In [36]:
comment = balenced_data["char_chat"]
target = balenced_data["target"]
char2idx = balenced_data["char2idx"]
idx2char= balenced_data["idx2char"]

In [38]:
a=[]
for i,k in zip(comment,target):
    a.append([i,k])

In [39]:
len(comment)

45427

In [40]:
len(target)

45427

In [41]:
def padding(token):
    token_post=token.copy()
    len_sentence=[ len(i) for i in token_post]
    max_sentence=max(len_sentence) #최대 문장길이 9591
    n=-1
    for i in token_post:
        n+=1
        k=len(token_post[n])
        while max_sentence-k>=0:
            token_post[n].append('<pad>')
            k+=1
    return token_post

In [42]:
comment=list(map(lambda x: padding(x),  [comment]))
comment=comment[0]

In [44]:
li1=[]
comment_list=[]
for i in comment:
    for k in i:
        li1.append(char2idx[k])
    
    comment_list.append(li1)
    li1=[]

In [66]:
X_train,X_test,y_train,y_test=train_test_split(comment_list,target,test_size=0.26,random_state=7,stratify=target)

In [67]:
len(X_train)

33615

In [49]:
model_word2vec=Word2Vec.load('../word2vec_model/char_model_200.model')

In [51]:
import numpy as np
import torch

embedding_matrix = []

for word in char2idx.keys():
    try:
        embedding_matrix.append(model_word2vec[word])
    except:
        embedding_matrix.append(np.zeros(200))

print(len(char2idx))
print(len(embedding_matrix))

embedding_matrix = torch.Tensor(embedding_matrix)

1962
1962


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [54]:
len(torch.FloatTensor(X_train[0]))

369

In [55]:
class LSTM_Clf(nn.Module):
    def __init__(self, input_size, embedding_size,num_class,num_layers,hidden_size,bidirectional,num_directions):
        
        super(LSTM_Clf,self).__init__()
        
        # 안에 하이퍼 파라미터의 종류들 선언
        
        self.num_layers=num_layers
        self.hidden_size=hidden_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.num_class=num_class
        self.num_directions=num_directions
        
        self.embedding = nn.Embedding.from_pretrained(embeddings=embedding_matrix)
                                    
   #     self.normal=torch.normal()
        self.lstm=nn.LSTM(input_size=embedding_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        bidirectional=bidirectional) # 해당 sequence의 앞 뒤를 모두 고려해서 반영
        
        self.linear=nn.Linear(in_features=hidden_size*num_directions,out_features=num_class)
        
    #    self.out=nn.Linear(num_class,1)

    
    def forward(self,inputs):
        
        hidden,cell=self.init_hidden(batch_size)
        
        embed=self.embedding(inputs)
        
    #    embed=self.normal(embed)
        
        out, _ =self.lstm(embed,(hidden,cell))
        
        out=self.linear(out[:, -1:, :].squeeze(1))
        
        return out
    
    
    def predict(self,inputs):
        
        hidden,cell = self.init_hidden(1)
        embed = self.embedding(inputs)
    #    embed=self.normal(embed)
        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        
        out, _ = self.lstm(embed, (hidden,cell))
        
        return self.linear(out[:, -1:, :]).squeeze(1)
    
    def init_hidden(self, batch_size):
        
        hidden=Variable(torch.zeros(self.num_layers*self.num_directions,batch_size, self.hidden_size)).to(device)
        cell=Variable(torch.zeros(self.num_layers*self.num_directions,batch_size, self.hidden_size)).to(device)
        
        return hidden,cell
    
    

In [68]:
class Dataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [69]:
train_data=Dataset(np.array(X_train),y_train)
test_data=Dataset(np.array(X_test),y_test)

In [70]:
train_loader=DataLoader(train_data,batch_size=27*5,shuffle=True)
test_loader=DataLoader(test_data,batch_size=1,shuffle=False)

In [71]:
batch_size=27*5

In [83]:
model = LSTM_Clf(input_size=len(char2idx),num_directions=2, embedding_size = 200,hidden_size=len(X_train[0]),
                num_layers=3,num_class=2,bidirectional=True).to(device)

loss_function = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.003,weight_decay=1e-5)

In [84]:
%%time

EPOCH=50
batch_size=27*5
for epoch in range(EPOCH):
    
    test_pred=[]
    test_true=[]
    train_loader=DataLoader(train_data,batch_size=27*5,shuffle=True)
    test_loader=DataLoader(test_data,batch_size=1,shuffle=False)
    
    for k,(X_batch,y_batch) in enumerate(train_loader):
        inputs=Variable(torch.LongTensor(X_batch)).to(device)
        targets=Variable(torch.LongTensor(y_batch)).to(device)
        
        y_pred=model(inputs)
        
        loss=loss_function(y_pred,targets)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (k+1) % 30==0 or k+1==89:
            print('epoch: [{}/{}] Step [{}/{}], Loss: {:.4f}'
                .format((epoch+1),EPOCH,(k+1),len(train_loader),loss))
    print('\n')        
    print('=====update finished!=====')
    
    for k,(X_batch,y_batch) in enumerate(test_loader):
        inputs=Variable(torch.LongTensor(X_batch)).to(device)
        targets=Variable(torch.LongTensor(y_batch)).to(device)
        
        y_pred=model.predict(inputs)
        y_pred=torch.max(y_pred,1)[1].item()
        
        test_pred.append(y_pred)
        test_true.append(targets.item())
        
    print('test_acc: %.3f' %(accuracy_score(y_true=test_true,y_pred=test_pred)))
    print('test_f1_score: %.3f' %(f1_score(y_true=test_true,y_pred=test_pred)))
    print('\n') 
        

print('------------------------------')

epoch: [1/50] Step [30/249], Loss: 0.5309
epoch: [1/50] Step [60/249], Loss: 0.5389
epoch: [1/50] Step [89/249], Loss: 0.5035
epoch: [1/50] Step [90/249], Loss: 0.6284
epoch: [1/50] Step [120/249], Loss: 0.4918
epoch: [1/50] Step [150/249], Loss: 0.5976
epoch: [1/50] Step [180/249], Loss: 0.5676
epoch: [1/50] Step [210/249], Loss: 0.5528
epoch: [1/50] Step [240/249], Loss: 0.5878


=====update finished!=====
test_acc: 0.777
test_f1_score: 0.000




/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


KeyboardInterrupt: 